In [1]:
# mkdir if not exist 'proposals_out_2023_07_20'
import os
if not os.path.exists('votes_out_2023_07_20'):
    os.makedirs('votes_out_2023_07_20')

In [4]:
# read in ../../DATA/deployments/snapshot_deployments.csv
import pandas as pd
df = pd.read_csv('../../DATA/proposals/snapshot_proposals.csv')
df.head()

,platform,platform_deployment_id,proposal_id,author,date,votes_count
0,snapshot,798777.eth,0xa3361c15ebb91e731146b652da16242384bd7a5679d8...,0x0242834979411FfCE95cDA74a5b00d0107a92d75,2023-07-20 16:52:11,1
1,snapshot,798777.eth,0x753a316234628e0e97bbc04c7513deec517fa99671fd...,0xCDCbFFa97EDaF82Abd6C02D2aB12D59D7A700a01,2023-07-20 16:34:38,1
2,snapshot,798777.eth,0x0e2ba26f4512e9ccc90491a1b9f64f395d6834a5eb2d...,0xF509B73e606A54281147711a51Cf8572BBE9987b,2023-07-20 16:12:38,1
3,snapshot,798777.eth,0x08fbbc7bbcfb0b972dcd6d5b4f60ee9a5be4470f0f43...,0x483a15A333eDC0A2C2D9Bbd986e6692A0FFA8471,2023-07-20 15:01:30,1
4,snapshot,798777.eth,0x61c7c68e5bd50cbb1ee45f3804eef83f0bc1be0d428f...,0x483a15A333eDC0A2C2D9Bbd986e6692A0FFA8471,2023-07-20 15:01:07,1


In [5]:
df.votes_count.describe()

count    1.246340e+05
mean     1.684988e+02
std      8.769128e+03
min      0.000000e+00
25%      1.000000e+00
50%      3.000000e+00
75%      1.800000e+01
max      2.765849e+06
Name: votes_count, dtype: float64

In [7]:
# sum of votes_count
df.votes_count.sum() / 5000
# we'll need this number of requests to get all votes

4200.1366

In [21]:
# remove the space_ids that have no proposals
df = df[df.votes_count > 0]
proposals_and_votes_count = df[['proposal_id', 'votes_count']].sort_values(by='votes_count', ascending=False)
# make these into a list of tuples
proposals_and_votes_count = list(proposals_and_votes_count.itertuples(index=False, name=None))
proposals_and_votes_count[:5]

[('0xda4f201a37ea08cf1892418e7b9e88f5687a68dbdc96c3ab22abaa1c7244648e',
  2765849),
 ('0xed5fe9d40c9e82a53c548d4458fdca91d6d26c12cc34303c205906b5e1854e29',
  470709),
 ('0xf1274081c9f4db1db77f30f21b53f61dde8716f8d3f2aac7367709bd4af369b2',
  450500),
 ('0x3cb9430062af89937887e9d359e4765de4f138c6bcea49ee66225a8fa998e97b',
  449253),
 ('0x85c675f303a69df297e1fdcf6dc2845366a8672362e946150ba4a4d7928cc8ea',
  446938)]

In [22]:
# make groups of rows where the proposals_count in the group sums to less than 5000
groups_of_proposal_ids_that_sum_to_less_than_5000 = []
while len(proposals_and_votes_count) > 0:
    group = []
    total = 0
    while total < 5000:
        row = proposals_and_votes_count.pop(0)
        group.append(row)
        total += row[1]
        if len(proposals_and_votes_count) == 0:
            break
    groups_of_proposal_ids_that_sum_to_less_than_5000.append(group)

groups_of_proposal_ids_that_sum_to_less_than_5000

[[('0xda4f201a37ea08cf1892418e7b9e88f5687a68dbdc96c3ab22abaa1c7244648e',
   2765849)],
 [('0xed5fe9d40c9e82a53c548d4458fdca91d6d26c12cc34303c205906b5e1854e29',
   470709)],
 [('0xf1274081c9f4db1db77f30f21b53f61dde8716f8d3f2aac7367709bd4af369b2',
   450500)],
 [('0x3cb9430062af89937887e9d359e4765de4f138c6bcea49ee66225a8fa998e97b',
   449253)],
 [('0x85c675f303a69df297e1fdcf6dc2845366a8672362e946150ba4a4d7928cc8ea',
   446938)],
 [('0x9f0e54337d515a9daa2e9cc05580b1993ebb18e2ebc465cf3ee6e777df369e2b',
   371130)],
 [('0x108a9e597560c4f249cd8be23acd409059fcd17bb2290d69a550ac2232676e7d',
   369729)],
 [('0x5280241b4ccc9b7c5088e657a714d28fa89bd5305a1ff0abf0736438c446ae98',
   365538)],
 [('0xcbefd708a0b64d9e9d634e5d7f57c5056b59285b6a5ba18492cfea4db6a53104',
   338494)],
 [('0x6b703b90d3cd1f82f7c176fc2e566a2bb79e8eb6618a568b52a4f29cb2f8d57b',
   264249)],
 [('0x04df8aafa123cf995b4ef28e0d28c4987ae09a41a4396235bd9c7368b05fdc80',
   225194)],
 [('0xda01dacad76bf6559fcd16cf947f23dd4ee3459e1d08aa6

In [13]:
# how many proposals have more than 5000 votes?
len(df[df.votes_count > 5000])

537

In [15]:
len(groups_of_proposal_ids_that_sum_to_less_than_5000)

1608

In [23]:
# make into a list of strings for the graphql query
groups_of_proposal_ids_that_sum_to_less_than_5000_strings = []
for group in groups_of_proposal_ids_that_sum_to_less_than_5000:
    group_of_space_ids = [str(x[0]) for x in group]
    query_string = ", ".join([f'"{sid}"' for sid in group_of_space_ids])
    groups_of_proposal_ids_that_sum_to_less_than_5000_strings.append(query_string)

groups_of_proposal_ids_that_sum_to_less_than_5000_strings[0]

'"0xda4f201a37ea08cf1892418e7b9e88f5687a68dbdc96c3ab22abaa1c7244648e"'

In [24]:
def make_query(proposal_ids: str, skip: int) -> str:
    first = 1000
    """
    * platform
    * platform_deployment_id
    * proposal_id
    * vote_id
    * voter -- the id of the person who made the vote
    * date -- this should be as a pandas date, not as an epoch
    * choice -- this should be a string as this is not always true/false
    * weight -- the vote weight
    """
    base = """
       query {
         votes(where: {proposal_in: [%s]}, first: %s, skip: %s, orderDirection: desc, orderBy: "created") {
            space {
              id
            }
            proposal {
              id
            }
            id
            voter
            created
            vp_by_strategy
            vp
          }
        }
    """
    return base % (proposal_ids, first, skip)


In [25]:
import requests
import backoff
URL = "https://hub.snapshot.org/graphql"


@backoff.on_exception(backoff.expo,
                      requests.exceptions.RequestException)
def query_snapshot(query):
    r = requests.post(URL, json={'query': query})
    return r.json()

In [33]:
import json
# TODO: continue refactor here!
for count, space_id_group in enumerate(groups_of_space_ids_that_sum_to_less_than_5000_strings):
    # make pagination logic
    skip = 0
    has_more = True
    while has_more:
        filename = f'proposals_out_2023_07_20/proposals_{count}_{skip}.json'
        next_proposals_filename = f'proposals_out_2023_07_20/proposals_{count}_{skip + 1000}.json'
        if os.path.exists(next_proposals_filename):
            print(f'next file exists: {next_proposals_filename}')
            skip += 1000
            continue
        next_group_filename = f'proposals_out_2023_07_20/proposals_{count + 1}_0.json'
        if os.path.exists(next_group_filename):
            print(f'next group file exists: {next_group_filename}')
            has_more = False
            continue
        query = make_query(space_id_group, skip)
        result = query_snapshot(query)
        proposals = result['data']['proposals']
        print('got', len(proposals), 'proposals for', count, skip)
        # save proposals to file in the output dir
        with open(filename, 'w') as f:
            json.dump(proposals, f)
        if len(proposals) == 1000:
            skip += 1000
        else:
            has_more = False


got 1000 proposals for 0 0
got 1000 proposals for 0 1000
got 1000 proposals for 0 2000
got 1000 proposals for 0 3000
got 1000 proposals for 0 4000
got 624 proposals for 0 5000
got 1000 proposals for 1 0
got 1000 proposals for 1 1000
got 1000 proposals for 1 2000
got 1000 proposals for 1 3000
got 749 proposals for 1 4000
got 1000 proposals for 2 0
got 1000 proposals for 2 1000
got 1000 proposals for 2 2000
got 1000 proposals for 2 3000
got 769 proposals for 2 4000
got 1000 proposals for 3 0
got 1000 proposals for 3 1000
got 1000 proposals for 3 2000
got 1000 proposals for 3 3000
got 1000 proposals for 3 4000
got 257 proposals for 3 5000
got 1000 proposals for 4 0
got 1000 proposals for 4 1000
got 1000 proposals for 4 2000
got 1000 proposals for 4 3000
got 1000 proposals for 4 4000
got 140 proposals for 4 5000
got 1000 proposals for 5 0
got 1000 proposals for 5 1000
got 1000 proposals for 5 2000
got 1000 proposals for 5 3000
got 1000 proposals for 5 4000
got 5 proposals for 5 5000
got 10